In [ ]:
import re
import time
import requests
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver

In [ ]:
# WeekList - 코드, 요일, 평점
code, week, rate = [], [], []

start = time.time()

for i in ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']:
    resp = requests.get('https://comic.naver.com/webtoon/weekdayList?week='+i)
    beso = BeautifulSoup(resp.text,'html.parser')
    
    container = beso.select('ul.img_list li')
    for cont in container:
        code.append(re.sub(r'[^0-9]', '', str(cont.select_one('dt'))[:45]))
        week.append(i)
        rate.append(float(cont.select_one('div.rating_type strong').text))

print(f"{time.time() - start:.0f} s")
        
# 웹툰 list - 제목, 작가, 줄거리, 장르1, 장르2, 연령대, 최고 회차 수
title, author, description, genre_1, genre_2, age, max_pg = [], [], [], [], [], [], []

for i in code:
    resp = requests.get('https://comic.naver.com/webtoon/list?titleId=' + str(i))
    beso = BeautifulSoup(resp.text, 'html.parser')
    
    title.append(beso.select('title')[0].text[:-10])
    author.append(beso.select('span.wrt_nm')[0].text[8:])
    description.append(beso.select('div.detail > p')[0].text)
    genre_1.append(beso.select('div.detail > p.detail_info > span.genre')[0].text.split(', ')[0])
    genre_2.append(beso.select('div.detail > p.detail_info > span.genre')[0].text.split(', ')[1])
    age.append(beso.select('div.detail > p.detail_info > span.age')[0].text)
    max_pg.append(int(re.sub(r'[^0-9]', '', beso.select_one('td.title').find('a')['href'][30:])))

df_list = pd.DataFrame({'code': code, 'week': week, 'rate': rate, 'title': title, 'author': author,
                        'description': description, 'genre_1': genre_1, 'genre_2': genre_2, 'age': age, 'max_pg': max_pg})

print(f"{time.time() - start:.0f} s")

In [ ]:
# 웹툰 detail - 코드, n회, 회별 제목, 회별 별점, 회별 별점 참여자 수, 회별 등록일, 회별 like 수

start = time.time()

code_a, ep_num, ep_title, rate_d, rate_d_cnt, date, likeit, comments_cnt = [], [], [], [], [], [], [], []

driver = webdriver.Chrome('./chromedriver', options=options)

for i in df_list['code']:
    
    for j in range(int(np.mean(df_list[df_list['code'] == i]['max_pg']))):
        raw = 'https://comic.naver.com/webtoon/detail?titleId=' + i + '&no=' + str(j + 1)
        driver.get(raw)
        
        code_a.append(i)
        ep_num.append(j + 1)
        ep_title.append(driver.find_elements_by_xpath('//div[@class="tit_area"]/div/h3')[0].text)
        rate_d.append(float(driver.find_elements_by_xpath('//span[@id="topPointTotalNumber"]/strong')[0].text))
        rate_d_cnt.append(int(driver.find_elements_by_xpath('//span[@class="pointTotalPerson"]/em')[0].text))
        date.append(driver.find_elements_by_xpath('//dd[@class="date"]')[0].text)
        likeit.append(int(driver.find_elements_by_xpath('//em[@class="u_cnt"]')[0].text.replace(',', '')))
        
        driver.switch_to.frame('commentIframe')
        
        comments_cnt.append(int(driver.find_elements_by_css_selector('.u_cbox_count')[0].text.replace(',', '')))
    
    print(f"{time.time() - start:.0f} s")
    
df_detail = pd.DataFrame({'code_a': code_a, 'ep_num': ep_num, 'ep_title': ep_title, 'rate_d': rate_d, 'rate_d_cnt': rate_d_cnt,
                          'date': date, 'likeit': likeit, 'comments_cnt': comments_cnt})

print(f"{time.time() - start:.0f} sss")

In [ ]:
driver = webdriver.Chrome('./chromedriver')
code_b, ep_num, comments = [], [], []

start = time.time()

for i in df_test['code']:
    
    for j in range(int(df_test[df_test['code'] == i]['max_pg'])):
        
        start = time.time()
        
        raw = 'https://comic.naver.com/webtoon/detail?titleId=' + i + '&no=' + str(j + 1)
        driver.get(raw)
        driver.switch_to.frame('commentIframe')
        driver.find_element_by_xpath('//li[@role="presentation"][2]/a').click()
        
        try:
            while True:
                
                for k in range(15):
                    code_b.append(i)
                    ep_num.append(j + 1)
                    comments.append(driver.find_elements_by_css_selector('.u_cbox_text_wrap')[k].text)

                for l in range(1, 10):
                    driver.find_element_by_xpath('//div[@class="u_cbox_page_wrap"]/a[@class="u_cbox_page"][' + str(l) + ']').click()
                    time.sleep(1)
                    
                    for m in range(15):
                        code_b.append(i)
                        ep_num.append(j + 1)
                        comments.append(driver.find_elements_by_css_selector('.u_cbox_text_wrap')[m].text)
                        
                driver.find_element_by_xpath('//div[@class="u_cbox_page_wrap"]/a[@class="u_cbox_next"]').click()
                time.sleep(1)
                
        except:
            pass
        
        print(f"{time.time() - start:.0f} s")

print(f"{time.time() - start:.0f} sss")